In [1]:
from dlisio import dlis
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

pd.set_option('display.max_rows', 500)

In [2]:
import dlisio
dlisio.__version__

'1.0.2'

In [3]:
f, *tail = dlis.load('Videos/ConCu_R03_Sta022_MDT_EDTA (1).dlis')

In [4]:
print(f)
print(tail)

LogicalFile(ConCu_R03_Sta022_MDT_EDTA)
[]


In [9]:
f.describe()

------------
Logical File
------------
Description : LogicalFile(ConCu_R03_Sta022_MDT_EDTA)
Frames      : 6
Channels    : 184

Known objects
--
CHANNEL                 : 184
FRAME                   : 6
TOOL                    : 10
FILE-HEADER             : 1
CALIBRATION-MEASUREMENT : 117
CALIBRATION-COEFFICIENT : 65
CALIBRATION             : 21
PARAMETER               : 567
PROCESS                 : 6
ORIGIN                  : 1
EQUIPMENT               : 35

Unknown objects
--
440-CHANNEL                 : 166
440-GROUP                   : 17
440-HZVERSIONINFO           : 2
440-LIS-INPU                : 166
440-OP-TOOL                 : 10
440-FILE                    : 1
440-CALIBRATION-COEFFICIENT : 65
440-ZONE                    : 1056
440-PASS                    : 1
440-PARAMETER               : 434


In [11]:
origin, *origin_tail = f.origins
print(len(origin_tail))

0


In [13]:
origin.describe()

------
Origin
------
name   : LAP-549A27AD-AA61-44D2-93D6-4957780FA821
origin : 39
copy   : 0

Logical file ID          : ConCu_R03_Sta022_MDT_EDTA
File set name and number : ARI / 996129507
File number and type     : 1 / CUSTOMER

Field                   : Wildcat
Well (id/name)          : 0410720283-00 / TCCSP OBS-1
Produced by (code/name) : 440 / Schlumberger
Produced for            : TCCSP LLC
Order number            : ARI MDT
Run number              : 3
Descent number          : 0
Created                 : 2024-10-02 21:48:37.092000

Created by              : HZ-OP:Wireline, (version: 13.0.221437.3100)
Other programs/services : DEPTH_DEVICE:Depth Measuring Device
                          MRPS-D:Single Probe, Not Sealed Bulk Head
                          TENSION_DEVICE:Tension Device
                          MRPO-XC:High Performance Pumpout
                          MRHY-HXB:High Performance Hydraulic Module
                          LOGGING_CABLE:Logging Cable
                 

In [15]:
for frame in f.frames:
    
    # Search through the channels for the index and obtain the units
    for channel in frame.channels:
        if channel.name == frame.index:
            depth_units = channel.units
    
    print(f'Frame Name: \t\t {frame.name}')
    print(f'Index Type: \t\t {frame.index_type}')
    print(f'Depth Interval: \t {frame.index_min} - {frame.index_max} {depth_units}')
    print(f'Depth Spacing: \t\t {frame.spacing} {depth_units}')
    print(f'Direction: \t\t {frame.direction}')
    print(f'Num of Channels: \t {len(frame.channels)}')
    print(f'Channel Names: \t\t {str(frame.channels)}')
    print('\n\n')

Frame Name: 		 600T
Index Type: 		 TIME
Depth Interval: 	 45568.200343221935 - 45568.20040326135 ms
Depth Spacing: 		 600.0 ms
Direction: 		 INCREASING
Num of Channels: 	 144
Channel Names: 		 [Channel(TIME), Channel(ETIM), Channel(TDEP), Channel(B1TR), Channel(B1TV), Channel(BAC1), Channel(BBP1), Channel(BCP1), Channel(BFR1), Channel(BHP1), Channel(BMB1), Channel(BMC1), Channel(BPC1), Channel(BPRCORR1), Channel(BQP1), Channel(BQS1G), Channel(BQT1), Channel(BRA1), Channel(BRF1), Channel(BRH1), Channel(BRQ1), Channel(BRR1), Channel(BRS1), Channel(BSC1), Channel(BSF1), Channel(BSG1), Channel(BSS1), Channel(BSW1), Channel(BVA1), Channel(DVP1), Channel(PDD1), Channel(PTRA1), Channel(PTT1), Channel(PTV1), Channel(RDVP1), Channel(HDC1), Channel(HET1), Channel(HHP1), Channel(HMC1), Channel(HMS1), Channel(HOL1), Channel(HPR1), Channel(HPV1), Channel(HS31), Channel(HSO1), Channel(POTCV), Channel(POTFR), Channel(POTMC), Channel(POUDCV), Channel(POUDDC), Channel(POUDET), Channel(POUDFR), Channel(

In [17]:
def summary_dataframe(object, **kwargs):
    # Create an empty dataframe
    df = pd.DataFrame()
    
    # Iterate over each of the keyword arguments
    for i, (key, value) in enumerate(kwargs.items()):
        list_of_values = []
        
        # Iterate over each parameter and get the relevant key
        for item in object:
            # Account for any missing values.
            try:
                x = getattr(item, key)
                list_of_values.append(x)
            except:
                list_of_values.append('')
                continue
        
        # Add a new column to our data frame
        df[value]=list_of_values
    
    # Sort the dataframe by column 1 and return it
    return df.sort_values(df.columns[0])
    

In [19]:
param_df = summary_dataframe(f.parameters, name='Name', long_name='Long Name', values='Value')

# Hiding people's names that may be in parameters.
# These two lines can be commented out to show them
mask = param_df['Name'].isin(['R8', 'RR1', 'WITN', 'ENGI'])
param_df = param_df[~mask]

param_df

,Name,Long Name,Value
269,A,Constant A of the Archie Formation Factor - Po...,[1.0]
268,ACTIVESERV,Primary ACTive Service,[]
115,ACTIVESERV1,Additional ACTive Service 1,[]
108,ACTIVESERV2,Additional ACTive Service 2,[]
129,ACTIVESERV3,Additional ACTive Service 3,[]
...,...,...,...
84,WSAL,Water Salinity,[-999.25]
370,WSAL_SPRI,Water Salinity,[-999.25]
127,WVIS,Water Viscosity,[0.5]
151,ZMUD,Acoustic Impedance of Mud,[1.4800000190734863]


In [21]:
channels = summary_dataframe(f.channels, name='Name', long_name='Long Name', dimension='Dimension', units='Units', frame='Frame')
channels

,Name,Long Name,Dimension,Units,Frame
150,50V,50 V Supply,[1],V,Frame(600T)
151,AZ_SL_EDTC,,[1],ft/s2,Frame(600T)
18,B1TR,MRPS 1 Resistivity Cell Temperature,[1],degF,Frame(600T)
19,B1TV,MRPS 1 Strain Gauge Temperature,[1],degF,Frame(600T)
20,BAC1,MRPS 1 Acquisition Command Echo,[1],,Frame(600T)
21,BBP1,MRPS 1 Quartz Gauge Nb_Prime Signal Component,[1],,Frame(600T)
22,BCP1,MRPS 1 Quartz Gauge Nc_Prime Signal Component,[1],,Frame(600T)
23,BFR1,MRPS 1 Flowline Fluid Resistivity,[1],ohm.m,Frame(600T)
24,BHP1,MRPS 1 Hydraulic Pressure,[1],psi,Frame(600T)
181,BHPR,Borehole Pressure,[1],psi,Frame(2000T)


In [27]:
tools = summary_dataframe(f.tools, name='Name', description='Description')
tools

,Name,Description
7,EDTA-A,Enhanced Downhole Tool Bus - Version A
8,EDTCB,Enhanced Digital Telemetry Cartridge - Version B
2,HY_1,Hydraulic Module (MRHY) 1
4,IFA_1,InSitu Fluid Analyzer 1
9,LEHQT,"Logging Equipment Head - QT, 3-3/8 inch 31 pin..."
0,MDT-BN,MDT Bottom Nose
5,MS_1,MDT Multi-Sample (MRMS) 1
6,PC,Power Cartridge
3,POUD,Dual Up-down Pumpout Module (MRPOUD)
1,PS_1,Single Probe Module (MRPS) 1


In [29]:
dsstb = f.object('TOOL', 'DSSTB')
dsstb_params = summary_dataframe(dsstb.parameters, name='Name', long_name='Long Name', values='Values')
dsstb_params

ValueError: Object not found: type=TOOL, name=DSSTB, origin='any', copynumber='any'

In [31]:
frame1 = f.object('FRAME','60B')

ValueError: Object not found: type=FRAME, name=60B, origin='any', copynumber='any'

In [33]:
curves = frame1.curves()

depth = curves['TDEP'] * 0.00254
dtco = curves['DTCO']
dtsm = curves['DTSM']
stc_mono = curves['SPR4']
wf_mono = curves['PWF4']

print(f'{depth.min()} - {depth.max()}')

NameError: name 'frame1' is not defined